In [2]:
import pandas as pd
from fuzzywuzzy import fuzz                         # /Users/haresh.sam/Downloads/dax_query_dashboard (1).xlsx
import numpy as np
import time
import tensorflow_hub as hub

def load_data(file_path):
    return pd.read_excel(file_path).head(100)

def lowercase_all(df):
    return df.applymap(lambda x: x.lower() if isinstance(x, str) else x)

def generate_embeddings(model, df):
    embeddings = {}
    for column in df.columns:
        column_embeddings = model(df[column].astype(str).tolist()).numpy()
        embeddings[column] = column_embeddings
    return embeddings

def compute_cosine_similarity(query_embedding, entry_embeddings):
    similarities = np.dot(entry_embeddings, query_embedding.T)
    return similarities.max()

def calculate_similarities(df, query, model):
    query_embedding = model([query]).numpy()
    results = []
    embeddings = generate_embeddings(model, df)
    
    for index, row in df.iterrows():
        row_similarities = []
        for column in df.columns:
            entry_embeddings = embeddings[column][index].reshape(1, -1)
            semantic_similarity = compute_cosine_similarity(query_embedding, entry_embeddings)
            
            # Fuzzy logic match
            fuzzy_similarity = fuzz.ratio(query, str(row[column]))
            fuzzy_similarity /= 100  # Normalize fuzzy score to [0, 1] range
            
            # Combine both similarities
            combined_similarity = (semantic_similarity + fuzzy_similarity) / 2
            row_similarities.append((column, combined_similarity))
        
        max_similarity = max(row_similarities, key=lambda x: x[1])[1]
        results.append((index, max_similarity))
    
    return results

def find_best_match(similarities):
    best_match = max(similarities, key=lambda x: x[1], default=None)
    return best_match

def process_data(file_path, query):
    df = load_data(file_path)
    df = lowercase_all(df)
    model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
    similarities = calculate_similarities(df, query, model)
    best_match = find_best_match(similarities)
    return best_match, df

def main():
    file_path = input("Enter the path to the Excel file: ")
    user_query = input("Enter the query to match: ")
    start_time = time.time()
    best_match, df = process_data(file_path, user_query)
    
    if best_match is not None:
        index, similarity = best_match
        print("File path:", file_path)
        print("Query given:", user_query)
        print("Best Match Found at index:", index)
        print("Similarity Score:", similarity)
        print(df.iloc[index])
    else:
        print("No match found")
    
    end_time = time.time()
    runtime = end_time - start_time
    print(f"Runtime: {runtime} seconds")

if __name__ == "__main__":
    main()


/var/folders/54/v3l5y7rn66b76fdzslzssp3w0000gn/T/ipykernel_47024/553762819.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df.applymap(lambda x: x.lower() if isinstance(x, str) else x)


File path: /Users/haresh.sam/Downloads/dax_query_dashboard (1).xlsx
Query given: deploy disc cum act
Best Match Found at index: 0
Similarity Score: 0.44786313593387606
Project                                             thomas_complete
Discipline                                                 general 
Table                                        deployment- discipline
Type                                                        measure
Dax Name                                 cumulative actual mh -disc
DAX Expression    =\nvar c= calculate([monthly actual mh],filter...
DisplayFolder                                                   NaN
FormatString                                                    #,0
IsHidden                                                      False
Description                                                 overall
Name: 0, dtype: object
Runtime: 2.3523011207580566 seconds
